In [ ]:
# Code contribution: Reng
# Before collaborative filtering, do train_test split 

In [ ]:
!pip install turicreate 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 92.0 MB 13 kB/s 
     |████████████████████████████████| 322 kB 59.8 MB/s 
     |████████████████████████████████| 3.5 MB 38.2 MB/s 
     |████████████████████████████████| 86.4 MB 82 kB/s 
     |████████████████████████████████| 3.6 MB 41.9 MB/s 
     |████████████████████████████████| 18.3 MB 311 kB/s 
     |████████████████████████████████| 3.8 MB 51.5 MB/s 
     |████████████████████████████████| 449 kB 54.8 MB/s 
     |████████████████████████████████| 2.9 MB 46.8 MB/s 
     |████████████████████████████████| 50 kB 5.5 MB/s 
     |████████████████████████████████| 20.1 MB 1.3 MB/s 
  Created wheel for prettytable: filename=prettytable-0.7.2-py3-none-any.whl size=13714 sha256=53c94b3519cfcdb174ab99f87fb794392e1c5f8a0b476bf831e139c90fb463c7
  Stored in directory: /root/.cache/pip/wheels/b2/7f/f6/f180315b584f00445045ff1699b550fa895d09471337ce21c6


In [ ]:
from sklearn.model_selection import GroupShuffleSplit
import pandas as pd 
import turicreate as tc 
from turicreate import SFrame
from turicreate.toolkits.recommender.util import precision_recall_by_user


In [ ]:
import os
import os.path
from google.colab import drive

# mount Google Drive to /content/drive/My Drive/
if os.path.isdir("/content/drive/MyDrive/Web Mining"):
  print("Google Drive already mounted")
else:
  drive.mount('/content/drive',force_remount=True)  
os.chdir("/content/drive/MyDrive/Web Mining")

Google Drive already mounted


In [ ]:
users = pd.read_csv("./Abhay/users_grouped.csv", skipinitialspace=True)
print(users.head())
print(len(users))

   UserID  Cluster Gender  Age  Occupation  Group
0    6040        0      M   25           6      1
1    3961        0      M   25           1      1
2    3966        0      M   25           0      1
3    3968        0      M   25           2      1
4    1835        0      M   25          19      1
6040


In [ ]:
print(users["Group"].nunique())

596


In [ ]:
gs = GroupShuffleSplit(n_splits=2, test_size=0.2, random_state=42)
train_ix, test_ix = next(gs.split(users, groups=users["Group"]))

In [ ]:
train = users.loc[train_ix]
print(train) 
test = users.loc[test_ix]
print(test)
print(train['Group'].nunique()) # correct we have 80% users in train, i.e. also 80% groups are in train 

      UserID  Cluster Gender  Age  Occupation  Group
0       6040        0      M   25           6      1
1       3961        0      M   25           1      1
2       3966        0      M   25           0      1
3       3968        0      M   25           2      1
4       1835        0      M   25          19      1
...      ...      ...    ...  ...         ...    ...
6035    3904       19      M    1          10    596
6036     940       19      M    1          10    596
6037    2217       19      M    1          10    596
6038    5063       19      F    1          10    596
6039       1       19      F    1          10    579

[4826 rows x 6 columns]
      UserID  Cluster Gender  Age  Occupation  Group
20      3949        0      M   45           0      3
21      2629        0      M   25          11      3
22      3921        0      M   25          14      3
23      1538        0      M   25           3      3
24      3926        0      M   25           2      3
...      ...      ...

In [ ]:
ratings = pd.read_csv("./ratings_new.csv")
ratings = ratings.drop('Timestamp', axis=1) 
print(ratings)

        UserID  MovieID  Rating
0            1     1193       5
1            1      661       3
2            1      914       3
3            1     3408       4
4            1     2355       5
...        ...      ...     ...
997251    6040     1091       1
997252    6040     1094       5
997253    6040      562       5
997254    6040     1096       4
997255    6040     1097       4

[997256 rows x 3 columns]


In [ ]:
act_ratings = ratings.copy() 
act_ratings = act_ratings.loc[act_ratings['UserID'].isin(train['UserID'])]
act_ratings.reset_index(drop=True, inplace=True) # made sure the users are all in train! 
print(act_ratings)

        UserID  MovieID  Rating
0            1     1193       5
1            1      661       3
2            1      914       3
3            1     3408       4
4            1     2355       5
...        ...      ...     ...
787605    6040     1091       1
787606    6040     1094       5
787607    6040      562       5
787608    6040     1096       4
787609    6040     1097       4

[787610 rows x 3 columns]


In [ ]:
movies = pd.read_csv("./movie_final_full_dataset.csv")

In [ ]:
act_movies = movies.copy() 
act_movies = act_movies.loc[act_movies['MovieID'].isin(act_ratings['MovieID'])]
print(act_movies)

      MovieID                               Title  \
0           1                    Toy Story (1995)   
1           2                      Jumanji (1995)   
2           3             Grumpier Old Men (1995)   
3           4            Waiting to Exhale (1995)   
4           5  Father of the Bride Part II (1995)   
...       ...                                 ...   
3357     3948             Meet the Parents (2000)   
3358     3949          Requiem for a Dream (2000)   
3359     3950                    Tigerland (2000)   
3360     3951             Two Family House (2000)   
3361     3952               Contender, The (2000)   

                                           Genres  ImdbID  \
0     Adventure|Animation|Children|Comedy|Fantasy  114709   
1                      Adventure|Children|Fantasy  113497   
2                                  Comedy|Romance  113228   
3                            Comedy|Drama|Romance  114885   
4                                          Comedy  113041 

# COPY UNTIL HERE

In [ ]:
act_movies = act_movies[["MovieID","Title", "Genres"]]
print(act_movies)

      MovieID                               Title  \
0           1                    Toy Story (1995)   
1           2                      Jumanji (1995)   
2           3             Grumpier Old Men (1995)   
3           4            Waiting to Exhale (1995)   
4           5  Father of the Bride Part II (1995)   
...       ...                                 ...   
3357     3948             Meet the Parents (2000)   
3358     3949          Requiem for a Dream (2000)   
3359     3950                    Tigerland (2000)   
3360     3951             Two Family House (2000)   
3361     3952               Contender, The (2000)   

                                           Genres  
0     Adventure|Animation|Children|Comedy|Fantasy  
1                      Adventure|Children|Fantasy  
2                                  Comedy|Romance  
3                            Comedy|Drama|Romance  
4                                          Comedy  
...                                           ...  

In [ ]:
# items = movies 
movies_tc = SFrame(data = act_movies) 
movies_tc.head()

MovieID,Title,Genres
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy ...
2,Jumanji (1995),Adventure|Children|Fantasy ...
3,Grumpier Old Men (1995),Comedy|Romance
4,Waiting to Exhale (1995),Comedy|Drama|Romance
5,Father of the Bride PartII (1995) ...,Comedy
6,Heat (1995),Action|Crime|Thriller
7,Sabrina (1995),Comedy|Romance
8,Tom and Huck (1995),Adventure|Children
9,Sudden Death (1995),Action
10,GoldenEye (1995),Action|Adventure|Thriller


In [ ]:
# actions = ratings 
ratings_tc = SFrame(data = act_ratings)
ratings_tc.head() 

UserID,MovieID,Rating
1,1193,5
1,661,3
1,914,3
1,3408,4
1,2355,5
1,1197,3
1,1287,5
1,2804,5
1,594,4
1,919,4


In [ ]:
act_train = train.copy()
act_train = train.drop(columns=["Cluster", "Group"])
users_tc = SFrame(data = act_train)
users_tc

UserID,Gender,Age,Occupation
6040,M,25,6
3961,M,25,1
3966,M,25,0
3968,M,25,2
1835,M,25,19
915,M,25,7
3973,M,25,7
913,M,25,0
3956,M,25,0
3974,M,35,0


In [ ]:
# not important
# ratings.tc.groupby('UserID', [tc.aggregate.COUNT]).sort("Count", ascending = False) 
movies_tc.show()

Materializing SFrame

In [ ]:
ratings_tc.show()

Materializing SFrame

In [ ]:
users_tc.show()

Materializing SFrame

In [ ]:
# train_tc, valid_tc = tc.recommender.util.random_split_by_user(ratings.tc, 'UserID', 'MovieID', item_test_proportion=0.2)

high_rated_data = ratings_tc[ratings_tc["Rating"] >= 4]
low_rated_data = ratings_tc[ratings_tc["Rating"] < 4]
train_tc_1, valid_tc = tc.recommender.util.random_split_by_user(
                                    high_rated_data, user_id='UserID', item_id='MovieID', item_test_proportion=0.2)
train_tc = train_tc_1.append(low_rated_data)

In [ ]:
# do chosen best by TC
chosen_model = tc.recommender.create(train_tc, 'UserID', 'MovieID', target='Rating',ranking= True)
# we can see that TC chose to use ranking_factorization_recommender 
# uses SGD 
#  Final objective value: 1.12309
#  Final training RMSE: 0.783743
# still output RMSE, we want to have precision/recall for ranks! 
# we want to test other models - to have more control over the process

Preparing data set.

Data has 768980 observations with 4826 users and 3362 items.

Data prepared in: 0.715117s

Training ranking_factorization_recommender for recommendations.

+--------------------------------+--------------------------------------------------+----------+

...

+---------+--------------+-------------------+-----------------------+-------------+

Optimization Complete: Maximum number of passes through the data reached.

Computing final objective value and training RMSE.

Final objective value: 1.12309

Final training RMSE: 0.783743

In [ ]:
# evaluate - model without user data, chosen by RC
chosen_model.evaluate(valid_tc)
#  'rmse_overall': 1.1949886462470833


Precision and recall summary statistics by cutoff
+--------+---------------------+---------------------+
| cutoff |    mean_precision   |     mean_recall     |
+--------+---------------------+---------------------+
|   1    | 0.32560483870967727 | 0.02108903642516187 |
|   2    | 0.28528225806451607 | 0.03642440974310098 |
|   3    | 0.27889784946236523 | 0.05326763150952403 |
|   4    |  0.2653729838709678 | 0.06890107837345123 |
|   5    | 0.25504032258064535 | 0.08288325691539035 |
|   6    | 0.24680779569892475 | 0.09697580958618354 |
|   7    | 0.24135944700460818 | 0.10950877227932894 |
|   8    | 0.23500504032258066 | 0.12006431258886516 |
|   9    | 0.22636648745519708 | 0.12915231074848846 |
|   10   | 0.22056451612903227 | 0.13953308197242023 |
+--------+---------------------+---------------------+
[10 rows x 3 columns]


Overall RMSE: 1.1969060511282292

Per User RMSE (best)
+--------+---------------------+-------+
| UserID |         rmse        | count |
+--------+--------

{'precision_recall_by_user': Columns:
 	UserID	int
 	cutoff	int
 	precision	float
 	recall	float
 	count	int
 
 Rows: 17856
 
 Data:
 +--------+--------+--------------------+---------------------+-------+
 | UserID | cutoff |     precision      |        recall       | count |
 +--------+--------+--------------------+---------------------+-------+
 |   5    |   1    |        1.0         | 0.06666666666666667 |   15  |
 |   5    |   2    |        0.5         | 0.06666666666666667 |   15  |
 |   5    |   3    | 0.3333333333333333 | 0.06666666666666667 |   15  |
 |   5    |   4    |        0.5         | 0.13333333333333333 |   15  |
 |   5    |   5    |        0.4         | 0.13333333333333333 |   15  |
 |   5    |   6    | 0.3333333333333333 | 0.13333333333333333 |   15  |
 |   5    |   7    | 0.2857142857142857 | 0.13333333333333333 |   15  |
 |   5    |   8    |        0.25        | 0.13333333333333333 |   15  |
 |   5    |   9    | 0.2222222222222222 | 0.13333333333333333 |   15  |
 | 

In [ ]:
side_data_model = tc.recommender.create(train_tc, 'UserID', 'MovieID', target='Rating', user_data=users_tc, ranking=True)
      #  Final objective value: 1.79761
      #  Final training RMSE: 1.02456

Preparing data set.

Data has 768980 observations with 4826 users and 3362 items.

Data prepared in: 0.734753s

Training ranking_factorization_recommender for recommendations.

+--------------------------------+--------------------------------------------------+----------+

...

+---------+--------------+-------------------+-----------------------+-------------+

Optimization Complete: Maximum number of passes through the data reached.

Computing final objective value and training RMSE.

Final objective value: 1.79761

Final training RMSE: 1.02456

In [ ]:
side_data_model.evaluate(valid_tc)
#  'rmse_overall': 1.2066669820367588}


Precision and recall summary statistics by cutoff
+--------+---------------------+----------------------+
| cutoff |    mean_precision   |     mean_recall      |
+--------+---------------------+----------------------+
|   1    | 0.18548387096774194 | 0.011276281760261648 |
|   2    | 0.18548387096774194 | 0.02088620772856979  |
|   3    | 0.18313172043010745 | 0.030093024596332927 |
|   4    |  0.1761592741935484 | 0.04044907272540124  |
|   5    | 0.17419354838709672 | 0.05092531096686008  |
|   6    | 0.16817876344086027 | 0.05899691557814078  |
|   7    | 0.16071428571428564 | 0.06627962406577922  |
|   8    | 0.15688004032258063 | 0.07272524136975646  |
|   9    | 0.15221774193548382 | 0.07991525456666114  |
|   10   | 0.14879032258064512 | 0.08662487245185217  |
+--------+---------------------+----------------------+
[10 rows x 3 columns]


Overall RMSE: 1.2066669820367588

Per User RMSE (best)
+--------+----------------------+-------+
| UserID |         rmse         | count |
+-

{'precision_recall_by_user': Columns:
 	UserID	int
 	cutoff	int
 	precision	float
 	recall	float
 	count	int
 
 Rows: 17856
 
 Data:
 +--------+--------+-----------+--------+-------+
 | UserID | cutoff | precision | recall | count |
 +--------+--------+-----------+--------+-------+
 |  3973  |   1    |    0.0    |  0.0   |   17  |
 |  3973  |   2    |    0.0    |  0.0   |   17  |
 |  3973  |   3    |    0.0    |  0.0   |   17  |
 |  3973  |   4    |    0.0    |  0.0   |   17  |
 |  3973  |   5    |    0.0    |  0.0   |   17  |
 |  3973  |   6    |    0.0    |  0.0   |   17  |
 |  3973  |   7    |    0.0    |  0.0   |   17  |
 |  3973  |   8    |    0.0    |  0.0   |   17  |
 |  3973  |   9    |    0.0    |  0.0   |   17  |
 |  3973  |   10   |    0.0    |  0.0   |   17  |
 +--------+--------+-----------+--------+-------+
 [17856 rows x 5 columns]
 Note: Only the head of the SFrame is printed.
 You can use print_rows(num_rows=m, num_columns=n) to print more rows and columns.,
 'precisio

In [ ]:
# print out side data 

In [ ]:
side_data_model

Class                            : RankingFactorizationRecommender

Schema
------
User ID                          : UserID
Item ID                          : MovieID
Target                           : Rating
Additional observation features  : 0
User side features               : ['UserID', 'Gender', 'Age', 'Occupation']
Item side features               : []

Statistics
----------
Number of observations           : 768980
Number of users                  : 4826
Number of items                  : 3362

Training summary
----------------
Training time                    : 98.2066

Model Parameters
----------------
Model class                      : RankingFactorizationRecommender
num_factors                      : 32
binary_target                    : 0
side_data_factorization          : 1
solver                           : auto
nmf                              : 0
max_iterations                   : 25

Regularization Settings
-----------------------
regularization                   : 0.0

In [ ]:
# do baseline - popularity_recommender 
# check if the difference is huge - if observation data is very sparse, 
# then no models will do better than this baseline (not good)
# dont do this
# change name since we are not doing pop (item_sim)
baseline = tc.recommender.item_similarity_recommender.create(train_tc, 'UserID', 'MovieID', target='Rating', user_data=users_tc)

Preparing data set.

Data has 768980 observations with 4826 users and 3362 items.

Data prepared in: 0.712033s

Training model from provided data.

Gathering per-item and per-user statistics.

...

| 3.52s                               | 100              | 3362            |

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 3.60768s

In [ ]:
baseline.evaluate(valid_tc)
# 3.7 


Precision and recall summary statistics by cutoff
+--------+---------------------+----------------------+
| cutoff |    mean_precision   |     mean_recall      |
+--------+---------------------+----------------------+
|   1    |  0.2933467741935484 | 0.025596209938617425 |
|   2    |  0.2862903225806451 | 0.04740661111250512  |
|   3    | 0.27688172043010745 |  0.0646120699251433  |
|   4    | 0.26638104838709686 |  0.0797614909056956  |
|   5    | 0.25161290322580654 | 0.09496370656492645  |
|   6    |  0.2399193548387096 | 0.10576104077187551  |
|   7    |  0.2299827188940092 | 0.11561702088612694  |
|   8    | 0.22379032258064516 | 0.12704429943006962  |
|   9    | 0.21527777777777793 | 0.13426143317948558  |
|   10   | 0.20826612903225808 | 0.14270541727379524  |
+--------+---------------------+----------------------+
[10 rows x 3 columns]


Overall RMSE: 4.392064499476559

Per User RMSE (best)
+--------+-------------------+-------+
| UserID |        rmse       | count |
+--------

{'precision_recall_by_user': Columns:
 	UserID	int
 	cutoff	int
 	precision	float
 	recall	float
 	count	int
 
 Rows: 17856
 
 Data:
 +--------+--------+---------------------+----------------------+-------+
 | UserID | cutoff |      precision      |        recall        | count |
 +--------+--------+---------------------+----------------------+-------+
 |  3973  |   1    |         1.0         | 0.058823529411764705 |   17  |
 |  3973  |   2    |         0.5         | 0.058823529411764705 |   17  |
 |  3973  |   3    |  0.3333333333333333 | 0.058823529411764705 |   17  |
 |  3973  |   4    |         0.25        | 0.058823529411764705 |   17  |
 |  3973  |   5    |         0.2         | 0.058823529411764705 |   17  |
 |  3973  |   6    |  0.3333333333333333 | 0.11764705882352941  |   17  |
 |  3973  |   7    | 0.42857142857142855 | 0.17647058823529413  |   17  |
 |  3973  |   8    |         0.5         | 0.23529411764705882  |   17  |
 |  3973  |   9    |  0.4444444444444444 | 0.23529411

In [ ]:
optimize = tc.ranking_factorization_recommender.create(train_tc, 'UserID', 'MovieID', target='Rating', user_data=users_tc, ranking_regularization=0.1, unobserved_rating_value = 1)
      #  Final objective value: 1.6475
      #  Final training RMSE: 1.04704

Preparing data set.

Data has 768980 observations with 4826 users and 3362 items.

Data prepared in: 0.742321s

Training ranking_factorization_recommender for recommendations.

+--------------------------------+--------------------------------------------------+----------+

...

+---------+--------------+-------------------+-----------------------+-------------+

Optimization Complete: Maximum number of passes through the data reached (hard limit).

Computing final objective value and training RMSE.

Final objective value: 1.6475

Final training RMSE: 1.04704

In [ ]:
optimize.evaluate(valid_tc)
#  'rmse_overall': 1.0776425796599836


Precision and recall summary statistics by cutoff
+--------+---------------------+----------------------+
| cutoff |    mean_precision   |     mean_recall      |
+--------+---------------------+----------------------+
|   1    | 0.21673387096774197 | 0.012797109764620788 |
|   2    |  0.1774193548387097 | 0.01921900111978292  |
|   3    |   0.17372311827957  | 0.027789936395157616 |
|   4    | 0.17237903225806459 | 0.038520052290639935 |
|   5    |  0.1693548387096774 | 0.04854367517623349  |
|   6    | 0.16750672043010742 |  0.0587571831692037  |
|   7    | 0.16172235023041476 | 0.06676394949915893  |
|   8    | 0.15738407258064518 |  0.0750539668088345  |
|   9    |  0.1521057347670252 |  0.0816900211882376  |
|   10   |  0.1475806451612903 | 0.08685594581828035  |
+--------+---------------------+----------------------+
[10 rows x 3 columns]


Overall RMSE: 1.0776425796599836

Per User RMSE (best)
+--------+----------------------+-------+
| UserID |         rmse         | count |
+-

{'precision_recall_by_user': Columns:
 	UserID	int
 	cutoff	int
 	precision	float
 	recall	float
 	count	int
 
 Rows: 17856
 
 Data:
 +--------+--------+-----------+--------+-------+
 | UserID | cutoff | precision | recall | count |
 +--------+--------+-----------+--------+-------+
 |  3973  |   1    |    0.0    |  0.0   |   17  |
 |  3973  |   2    |    0.0    |  0.0   |   17  |
 |  3973  |   3    |    0.0    |  0.0   |   17  |
 |  3973  |   4    |    0.0    |  0.0   |   17  |
 |  3973  |   5    |    0.0    |  0.0   |   17  |
 |  3973  |   6    |    0.0    |  0.0   |   17  |
 |  3973  |   7    |    0.0    |  0.0   |   17  |
 |  3973  |   8    |    0.0    |  0.0   |   17  |
 |  3973  |   9    |    0.0    |  0.0   |   17  |
 |  3973  |   10   |    0.0    |  0.0   |   17  |
 +--------+--------+-----------+--------+-------+
 [17856 rows x 5 columns]
 Note: Only the head of the SFrame is printed.
 You can use print_rows(num_rows=m, num_columns=n) to print more rows and columns.,
 'precisio

In [ ]:
# ranking factorization recommender

In [ ]:
# at last we want to fine tune to model such as num_factors and regularization 
# https://apple.github.io/turicreate/docs/api/generated/turicreate.recommender.ranking_factorization_recommender.RankingFactorizationRecommender.html

In [ ]:
tc.recommender.util.compare_models(valid_tc, [chosen_model, side_data_model, baseline, optimize], metric='precision_recall')

PROGRESS: Evaluate model M0

Precision and recall summary statistics by cutoff
+--------+---------------------+----------------------+
| cutoff |    mean_precision   |     mean_recall      |
+--------+---------------------+----------------------+
|   1    | 0.32560483870967744 | 0.021089036425161852 |
|   2    |  0.2852822580645162 | 0.036424409743100974 |
|   3    |  0.2788978494623658 | 0.05326763150952405  |
|   4    | 0.26537298387096764 |  0.0689010783734512  |
|   5    |  0.2550403225806452 | 0.08288325691539035  |
|   6    | 0.24680779569892472 | 0.09697580958618356  |
|   7    |  0.2413594470046082 | 0.10950877227932895  |
|   8    | 0.23500504032258066 | 0.12006431258886512  |
|   9    |  0.2263664874551972 |  0.1291523107484885  |
|   10   | 0.22056451612903233 | 0.13953308197242031  |
+--------+---------------------+----------------------+
[10 rows x 3 columns]

PROGRESS: Evaluate model M1

Precision and recall summary statistics by cutoff
+--------+---------------------+---

[{'precision_recall_by_user': Columns:
  	UserID	int
  	cutoff	int
  	precision	float
  	recall	float
  	count	int
  
  Rows: 17856
  
  Data:
  +--------+--------+--------------------+---------------------+-------+
  | UserID | cutoff |     precision      |        recall       | count |
  +--------+--------+--------------------+---------------------+-------+
  |   5    |   1    |        1.0         | 0.06666666666666667 |   15  |
  |   5    |   2    |        0.5         | 0.06666666666666667 |   15  |
  |   5    |   3    | 0.3333333333333333 | 0.06666666666666667 |   15  |
  |   5    |   4    |        0.5         | 0.13333333333333333 |   15  |
  |   5    |   5    |        0.4         | 0.13333333333333333 |   15  |
  |   5    |   6    | 0.3333333333333333 | 0.13333333333333333 |   15  |
  |   5    |   7    | 0.2857142857142857 | 0.13333333333333333 |   15  |
  |   5    |   8    |        0.25        | 0.13333333333333333 |   15  |
  |   5    |   9    | 0.2222222222222222 | 0.133333333

In [ ]:
# https://antonellocalamea.medium.com/introduction-to-product-recommender-with-apples-turi-create-7f9f02fd0063

In [ ]:
# 
results = optimize.recommend(k = 3700)
# it will make sures that it does not exist the number of movies not watched by each user! 

# todo: we should recommend on all test and training ????? 

recommendations finished on 1000/4826 queries. users per second: 686.133

recommendations finished on 2000/4826 queries. users per second: 689.518

recommendations finished on 3000/4826 queries. users per second: 692.899

recommendations finished on 4000/4826 queries. users per second: 699.668

In [ ]:
# # def join_titles(sframe,on):
# #     return sframe.join(right=movies_tc, on=on, how='inner')
# # results = join_titles(results,'MovieID')
# test = results.sort(['UserID','rank'], ascending=True)

In [ ]:
# test[test['UserID'] == 6].tail()

# # for user 3, 3644 rows 
# # for user 6, 3623 rows 

UserID,MovieID,score,rank
6,1590,2.642470050677616,3282
6,3689,2.6213223115354367,3283
6,2701,2.6208602702989765,3284
6,2153,2.6050075852125474,3285
6,3593,2.5820697397738286,3286
6,1556,2.526595126911957,3287
6,2450,2.484671924337704,3288
6,1998,2.4528008066862412,3289
6,1389,2.4254260045289584,3290
6,2950,2.335322082117159,3291


In [ ]:
# test2 = test[test['UserID'] == 3]
# test2['MovieID'].unique()

dtype: int
Rows: 3311
[3143, 2035, 1611, 2779, 118, 2057, 2847, 2459, 3270, 3365, 1236, 1657, 2925, 839, 2807, 3289, 3578, 3913, 2043, 3159, 2464, 2465, 2133, 2445, 2400, 567, 1794, 1029, 1504, 435, 1289, 1517, 554, 898, 2236, 2695, 840, 537, 2011, 3858, 526, 2846, 1480, 2231, 1874, 1416, 2634, 1990, 1420, 3158, 2084, 783, 257, 3637, 217, 2347, 232, 1854, 310, 3112, 2027, 998, 2440, 3808, 1387, 2276, 947, 49, 2207, 3509, 3634, 731, 1222, 2316, 2086, 3551, 424, 2736, 2953, 680, 13, 3537, 3439, 1437, 241, 225, 3880, 1991, 2668, 1950, 14, 2336, 2005, 2686, 3535, 3766, 511, 3138, 363, 78, ... ]

In [ ]:
results_df = results.to_dataframe()

In [ ]:
results_df['score'].describe()

count    1.545603e+07
mean     3.309635e+00
std      1.767194e-01
min      2.454613e+00
25%      3.194730e+00
50%      3.302747e+00
75%      3.425393e+00
max      4.510223e+00
Name: score, dtype: float64

In [ ]:
results.export_csv('./Final Data/Single-CF-train.csv')

In [ ]:
CFtest = pd.read_csv('./Final Data/Single-CF-train.csv')
CFtest.head()

,UserID,MovieID,score,rank
0,6040,1197,3.78695,1
1,6040,110,3.74904,2
2,6040,3578,3.64021,3
3,6040,356,3.63962,4
4,6040,3114,3.62049,5


In [ ]:
CFtest['score'].describe()
# CFtest['DataFrame Column'] = df['DataFrame Column'].replace(np.nan, 0)

count    1.545603e+07
mean     3.309635e+00
std      1.767194e-01
min      2.454610e+00
25%      3.194730e+00
50%      3.302750e+00
75%      3.425390e+00
max      4.510220e+00
Name: score, dtype: float64

In [ ]:
optimize.save("recommendations.model")
# model = tc.load_model("recommendations.model") # to use the model


Prediction for test set

In [ ]:
act_test = test.copy()
act_test = test.drop(columns=["Cluster", "Group"])
users_test_tc = SFrame(data = act_test)
users_test_tc

UserID,Gender,Age,Occupation
3949,M,45,0
2629,M,25,11
3921,M,25,14
1538,M,25,3
3926,M,25,2
937,M,25,15
3928,M,25,0
3929,M,25,5
3931,M,25,5
3932,M,25,7


In [ ]:
users_test_tc.show()

Materializing SFrame

In [ ]:
act_ratings_test = ratings.copy() 
act_ratings_test = act_ratings_test.loc[act_ratings_test['UserID'].isin(test['UserID'])]
act_ratings_test.reset_index(drop=True, inplace=True) # made sure the users are all in test! 
print(act_ratings_test)

        UserID  MovieID  Rating
0            2     1357       5
1            2     3068       4
2            2     1537       4
3            2      647       3
4            2     2194       4
...        ...      ...     ...
209641    6038     1387       2
209642    6038     2700       1
209643    6038     2716       3
209644    6038     3396       3
209645    6038     1079       5

[209646 rows x 3 columns]


In [ ]:
ratings_test_tc = SFrame(data = act_ratings_test)
print(ratings_test_tc.head()) 
ratings_test_tc.show()

+--------+---------+--------+
| UserID | MovieID | Rating |
+--------+---------+--------+
|   2    |   1357  |   5    |
|   2    |   3068  |   4    |
|   2    |   1537  |   4    |
|   2    |   647   |   3    |
|   2    |   2194  |   4    |
|   2    |   648   |   4    |
|   2    |   2268  |   5    |
|   2    |   2628  |   3    |
|   2    |   1103  |   3    |
|   2    |   2916  |   3    |
+--------+---------+--------+
[10 rows x 3 columns]



Materializing SFrame

In [ ]:
pred_test = optimize.predict(ratings_test_tc, new_user_data = users_test_tc)

https://medium.com/analytics-vidhya/building-a-recommendation-system-with-turi-create-938be5fea671

In [ ]:
pred_test_df = act_ratings_test.copy() 
pred_test_df['Rating'] = pred_test
pred_test_df

,UserID,MovieID,Rating
0,2,1357,3.650502
1,2,3068,3.589013
2,2,1537,3.635573
3,2,647,3.565670
4,2,2194,3.860406
...,...,...,...
209641,6038,1387,3.796086
209642,6038,2700,3.666036
209643,6038,2716,3.774065
209644,6038,3396,3.645155


In [ ]:
pred_test_df['Rating'].describe()

count    209646.000000
mean          3.412923
std           0.212479
min           2.640933
25%           3.258420
50%           3.385139
75%           3.538141
max           4.267178
Name: Rating, dtype: float64

In [ ]:
pred_test_df.to_csv('./Final Data/Single-CF-test.csv', index=False)